In [1]:
!pip install transformers
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+${cu102}.html
# !conda install pytorch-scatter -c rusty1s

     |████████████████████████████████| 2.5MB 5.4MB/s 
     |████████████████████████████████| 901kB 31.0MB/s 
     |████████████████████████████████| 3.3MB 33.8MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+.html
  Created wheel for torch-scatter: filename=torch_scatter-2.0.7-cp37-cp37m-linux_x86_64.whl size=271433 sha256=1e274221f336b942753cdc73ed700e1b4765b1e048fa1425a2a11696460738da
  Stored in directory: /root/.cache/pip/wheels/d4/47/a3/b454bae87fa5c1631826bc9b15285622675eac6baf4096b484
Successfully built torch-scatter


In [2]:
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('we love you'))"

2021-07-09 14:02:43.451004: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Downloading: 100% 629/629 [00:00<00:00, 472kB/s]
Downloading: 100% 268M/268M [00:08<00:00, 32.6MB/s]
Downloading: 100% 48.0/48.0 [00:00<00:00, 35.6kB/s]
Downloading: 100% 232k/232k [00:00<00:00, 4.53MB/s]
[{'label': 'POSITIVE', 'score': 0.9998704791069031}]


In [3]:
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd
import re
import torch
print(torch.__version__)

1.9.0+cu102


In [4]:
filter_list = ["of","to","who","is","the","in","it","be","not"]

In [5]:
def get_unique_words_from_df(df):
    tokens = []
    for col in df.columns:
        tokens += col.split()
        for c in df[col]:
            if type(c) == str and len(c) > 0:
                tokens += c.split()
    tokens = [t.lower() for t in tokens]
    tokens = [re.sub(r'[^A-Za-z0-9 ]+', '', t) for t in tokens]
    tokens = list(set(tokens))
    return tokens

In [6]:
def get_best_table(dfs,dfs_tokens,query):
    dfs_scores = []
    query_tokens = query.lower().split()
    query_tokens = [re.sub(r'[^A-Za-z0-9 ]+', '', t) for t in query_tokens]
    query_tokens = [t for t in query_tokens if t not in filter_list]
    print(query_tokens)
    for df,tokens in zip(dfs,dfs_tokens):
        total = 0
        score = 0
        for qt in query_tokens:
            if qt in tokens:
                score += 1
            total += 1
        
        if total > 0:
            dfs_scores.append(score/total)
        else:
            dfs_scores.append(0)
    
    max_score = max(dfs_scores)
    if max_score > 0:
        idx = dfs_scores.index(max_score)
        return idx
    else:
        return None


In [7]:
model_name = 'google/tapas-base-finetuned-wtq'
model = TapasForQuestionAnswering.from_pretrained(model_name)
tokenizer = TapasTokenizer.from_pretrained(model_name)

In [94]:
table1 = {'Actors': ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], 'Number of movies': ["87", "53", "69"], 'Test': ['34','54','36']}
df_table1 = pd.DataFrame.from_dict(table1)

table2 = {'Chairmen': ["Elon Musk", "Jeff Bezos", "Bill Gates", "Sergei Brin", "Mark Zuckerburg"], 'Big Companies': ["Tesla", "Amazon", "Microsoft", "Google", "Facebook"], 'Test': ['34','54','36','37','38']}
df_table2 = pd.DataFrame.from_dict(table2)

table3 = {"Datapoint":["Total Revenue","Total Profit or Loss","Gross Margin"],"2018":["105M","10M","95M"],"2017":["90M","5M","85M"]}
df_table3 = pd.DataFrame.from_dict(table3)

# df_table4 = df = pd.read_csv('/content/XboxOne_GameSales.csv', encoding = "ISO-8859-1")
table4 = {"Game":["Grand Theft Auto V","Call of Duty: Black Ops 3","MineCraft"],"Publisher":["Rockstar Games","Activision","Microsoft Studios"],"Genre":["Action","Shooter","Misc"]}
df_table4 = pd.DataFrame.from_dict(table4)


dfs = [df_table1,df_table2,df_table3, df_table4]

In [96]:
dfs_tokens = [get_unique_words_from_df(df) for df in dfs]
queries = [
    "What is the name of the first actor?", 
    "How many movies has George Clooney played in?", 
    "What is the total number of movies?",
    "Who is the Chairman of Google?",
    "What was the total revenue in 2017",
    "Gross Margin in 2018?",
    "What genre is Grand Theft Auto V",
    "Who is the Publisher of MineCraft"
]

In [97]:
q_idxs = []
for q in queries:
    idx = get_best_table(dfs,dfs_tokens,q)
    if idx is not None:
        q_idxs.append(idx)
    else:
        q_idxs.append(None)

['what', 'name', 'first', 'actor']
['how', 'many', 'movies', 'has', 'george', 'clooney', 'played']
['what', 'total', 'number', 'movies']
['chairman', 'google']
['what', 'was', 'total', 'revenue', '2017']
['gross', 'margin', '2018']
['what', 'genre', 'grand', 'theft', 'auto', 'v']
['publisher', 'minecraft']


In [98]:
unanswered_queries = []
qmap = {}
for idx, qidx in enumerate(q_idxs):
    if qidx == None:
        unanswered_queries.append(queries[idx])
    else:
        if qidx not in qmap:
            qmap[qidx] = []
        qmap[qidx].append(queries[idx])

In [99]:
id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3:"COUNT", 4: "*"}

In [100]:
for idx in qmap:
    df = dfs[idx]
    inputs = tokenizer(table=df, queries=qmap[idx], padding='max_length', return_tensors="pt")
    outputs = model(**inputs)
    predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs,
        outputs.logits.detach(),
        outputs.logits_aggregation.detach()
    )
    aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]
    answers = []
    for coordinates in predicted_answer_coordinates:
        if len(coordinates) == 1:
            # only a single cell:
            answers.append(df.iat[coordinates[0]])
        else:
            # multiple cells
            cell_values = []
            for coordinate in coordinates:
                cell_values.append(df.iat[coordinate])
            answers.append(", ".join(cell_values))
    print("")

    for query, answer, predicted_agg in zip(qmap[idx], answers, aggregation_predictions_string):
        print(query)
        if predicted_agg in ["NONE","COUNT"]:
            print("Predicted answer: " + answer)
        else:
            print("Predicted answer: " + predicted_agg + "(" + answer + ")")


How many movies has George Clooney played in?
Predicted answer: 69
What is the total number of movies?
Predicted answer: SUM(87, 53, 69)

Who is the Chairman of Google?
Predicted answer: Sergei Brin

What was the total revenue in 2017
Predicted answer: AVERAGE(90M)
Gross Margin in 2018?
Predicted answer: AVERAGE(95M)

What genre is Grand Theft Auto V
Predicted answer: Action
Who is the Publisher of MineCraft
Predicted answer: Microsoft Studios


In [63]:
df = pd.read_csv('/content/XboxOne_GameSales.csv', encoding = "ISO-8859-1")
df.head()

,Pos,Game,Year,Genre,Publisher,North America,Europe,Japan,Rest of World,Global
0,1,Grand Theft Auto V,2014.0,Action,Rockstar Games,4.70,3.25,0.01,0.76,8.72
1,2,Call of Duty: Black Ops 3,2015.0,Shooter,Activision,4.63,2.04,0.02,0.68,7.37
2,3,Call of Duty: WWII,2017.0,Shooter,Activision,3.75,1.91,0.00,0.57,6.23
3,4,Red Dead Redemption 2,2018.0,Action-Adventure,Rockstar Games,3.76,1.47,0.00,0.54,5.77
4,5,MineCraft,2014.0,Misc,Microsoft Studios,3.23,1.71,0.00,0.49,5.43
